<a href="https://colab.research.google.com/github/Icetiger1/python/blob/main/%D0%97%D0%B0%D0%BD%D1%8F%D1%82%D0%B8%D0%B5%2015.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get install openjdk-11-jdk-headless > /dev/null
!wget https://archive.apache.org/dist/spark/spark-3.2.1/spark-3.2.1-bin-hadoop2.7.tgz
!tar -xf /content/spark-3.2.1-bin-hadoop2.7.tgz
!pip install findspark
!pip install pyspark==3.2.1

In [ ]:
#переменные окружения
import os
os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-11-openjdk-amd64'
os.environ['SPARK_HOME'] = '/content/spark-3.2.1-bin-hadoop2.7'

#import

In [ ]:
#spark
import findspark
import pyspark
from pyspark import SparkContext
from pyspark.sql import SparkSession
findspark.init()

#spark session

In [ ]:
#создаем спарк сессию
spark = SparkSession.builder.config("spark.executor.memory", "8g")\
                            .config("spark.executor.cores", "3")\
                            .config("spark.cores.max", "3")\
                            .config("spark.driver.memory", "8g")\
                            .config("spark.dynamicAllocation.enabled", "True").getOrCreate()

#Зарузка данных

In [ ]:
!git clone https://github.com/Lasttrader/DA_repo

Cloning into 'DA_repo'...
remote: Enumerating objects: 63, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 63 (delta 1), reused 1 (delta 0), pack-reused 58
Receiving objects: 100% (63/63), 31.42 MiB | 10.67 MiB/s, done.
Resolving deltas: 100% (14/14), done.


In [ ]:
path = '/content/DA_repo/Gazprom_stocks/04_05.csv'

In [ ]:
#загружаем датасет
data_spark = spark.read.csv(path, inferSchema = True, header = True, sep = ';')
print(f'кол-во строк = {data_spark.count()}')
print(f'кол-во столбцов = {len(data_spark.dtypes)}')
data_spark.show(10)

кол-во строк = 246015
кол-во столбцов = 7
+----------+--------+------+------+------+------+------+
|      DATE|    TIME|  OPEN|   MAX|   MIN| CLOSE|VOLUME|
+----------+--------+------+------+------+------+------+
|05.01.2004|10:30:00|681.05| 685.0|681.05| 684.0|  4833|
|05.01.2004|10:31:00|684.79| 688.0|684.01| 686.0|  7025|
|05.01.2004|10:32:00| 685.0| 688.0| 685.0|685.12|  3956|
|05.01.2004|10:33:00|686.98| 687.0|686.02|686.02|  1105|
|05.01.2004|10:34:00| 687.0|688.68| 687.0| 688.0|  6270|
|05.01.2004|10:35:00| 688.5| 688.5|687.02| 687.9|   576|
|05.01.2004|10:36:00| 687.1| 687.1| 687.0| 687.0|  3330|
|05.01.2004|10:37:00|686.99|686.99| 686.0| 686.0|  2679|
|05.01.2004|10:38:00| 686.0| 686.5| 686.0| 686.0|  6227|
|05.01.2004|10:39:00| 686.5| 686.5| 686.0| 686.0|  1350|
+----------+--------+------+------+------+------+------+
only showing top 10 rows



In [ ]:
#кол-во строк в датасете
data_spark.count()

246015

In [ ]:
#кол-востолбцов
len(data_spark.dtypes)

7

In [ ]:
print(f'кол-во строк = {data_spark.count()}')
print(f'кол-во столбцов = {len(data_spark.dtypes)}')

кол-во строк = 246015
кол-во столбцов = 7


#Spark & SQL

In [ ]:
data_spark.registerTempTable("my_temp_sql_table") #временная таблица
#sql
query_data_spark = spark.sql(''' SELECT OPEN , MAX
                                  FROM my_temp_sql_table
                                  WHERE MAX = 688.0
                             ''')
query_data_spark.show(10)

/usr/local/lib/python3.10/dist-packages/pyspark/sql/dataframe.py:138: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn(


+------+-----+
|  OPEN|  MAX|
+------+-----+
|684.79|688.0|
| 685.0|688.0|
| 687.9|688.0|
|687.23|688.0|
| 687.9|688.0|
| 687.0|688.0|
| 687.4|688.0|
| 688.0|688.0|
| 687.8|688.0|
| 685.1|688.0|
+------+-----+
only showing top 10 rows



In [ ]:
values_list = [687.9, 687.8, 685.1]
#sql
query_data_spark = spark.sql(f''' SELECT OPEN, MAX, MIN,CLOSE, VOLUME
                                  FROM my_temp_sql_table
                                  WHERE OPEN IN {tuple(values_list)}
                             ''')
query_data_spark.show(10)

+-----+------+------+------+------+
| OPEN|   MAX|   MIN| CLOSE|VOLUME|
+-----+------+------+------+------+
|687.9| 687.9| 687.5| 687.5|  1018|
|687.9| 688.0| 687.9|687.99|    97|
|687.9| 688.0| 687.9| 687.9|  1631|
|687.9| 687.9|687.01| 687.7|  5894|
|687.8| 688.0| 686.5| 687.1|  6525|
|685.1| 688.0|685.01| 687.5| 16304|
|687.8|687.93| 687.8|687.93|  1645|
+-----+------+------+------+------+



#Spark to pandas

In [ ]:
my_df_pandas = query_data_spark.toPandas()
my_df_pandas.head() #pandas датафрейм

,OPEN,MAX,MIN,CLOSE,VOLUME
0,687.9,687.9,687.50,687.50,1018
1,687.9,688.0,687.90,687.99,97
2,687.9,688.0,687.90,687.90,1631
3,687.9,687.9,687.01,687.70,5894
4,687.8,688.0,686.50,687.10,6525


#Spark ML

In [ ]:
#особенности spark
1. обычный Х = my_df_pandas.drop('y', axis =1)
2. spark отдельная колонка вида X = [687.9,	687.9,	687.50,	687.50,	1018]

In [ ]:
from pyspark.ml.linalg import Vectors #для работыс признаками
from pyspark.ml.feature import MinMaxScaler, VectorAssembler #для масштабирования данных
from pyspark.ml.regression import LinearRegression, LinearRegressionModel
from pyspark.ml import Pipeline

#подготовка признаков

In [ ]:
#объединение колонок в один веткор
assembler = VectorAssembler(inputCols = ['OPEN','MAX', 'MIN', 'VOLUME'],
                            outputCol='vector_cols')
#minmaxscaler
scaler = MinMaxScaler(inputCol='vector_cols',
                      outputCol='vector_cols_scaled') #масштабирвоание
#pipeline
pipeline = Pipeline(stages = [assembler, scaler]) #параметры и шаги пайплайны
pipe_fit = pipeline.fit(data_spark)#обучение
dataset = pipe_fit.transform(data_spark) #добавялем к датасету новые колонки после пайпайна
dataset.show()

+----------+--------+------+------+------+------+------+--------------------+--------------------+
|      DATE|    TIME|  OPEN|   MAX|   MIN| CLOSE|VOLUME|         vector_cols|  vector_cols_scaled|
+----------+--------+------+------+------+------+------+--------------------+--------------------+
|05.01.2004|10:30:00|681.05| 685.0|681.05| 684.0|  4833|[681.05,685.0,681...|[0.0,4.5850947739...|
|05.01.2004|10:31:00|684.79| 688.0|684.01| 686.0|  7025|[684.79,688.0,684...|[0.00342493978882...|
|05.01.2004|10:32:00| 685.0| 688.0| 685.0|685.12|  3956|[685.0,688.0,685....|[0.00361724924220...|
|05.01.2004|10:33:00|686.98| 687.0|686.02|686.02|  1105|[686.98,687.0,686...|[0.00543045265982...|
|05.01.2004|10:34:00| 687.0|688.68| 687.0| 688.0|  6270|[687.0,688.68,687...|[0.00544876784585...|
|05.01.2004|10:35:00| 688.5| 688.5|687.02| 687.9|   576|[688.5,688.5,687....|[0.00682240679859...|
|05.01.2004|10:36:00| 687.1| 687.1| 687.0| 687.0|  3330|[687.1,687.1,687....|[0.00554034377604...|
|05.01.200

#X,y

In [ ]:
dataset_lr = dataset.select(['vector_cols_scaled', 'CLOSE'])
dataset_lr.show()

+--------------------+------+
|  vector_cols_scaled| CLOSE|
+--------------------+------+
|[0.0,4.5850947739...| 684.0|
|[0.00342493978882...| 686.0|
|[0.00361724924220...|685.12|
|[0.00543045265982...|686.02|
|[0.00544876784585...| 688.0|
|[0.00682240679859...| 687.9|
|[0.00554034377604...| 687.0|
|[0.00543961025284...| 686.0|
|[0.00453300854403...| 686.0|
|[0.00499088819494...| 686.0|
|[0.00453300854403...|686.01|
|[0.00526561598549...|685.53|
|[0.00410260167217...| 684.0|
|[0.00360809164919...| 685.0|
|[0.00292127217282...|684.53|
|[0.00318684237035...| 685.0|
|[0.00451469335799...| 686.0|
|[0.00488099707872...| 686.0|
|[0.00453300854403...| 686.9|
|[0.00533887672964...| 686.0|
+--------------------+------+
only showing top 20 rows



#train_test_split

In [ ]:
split = dataset_lr.randomSplit([0.8, 0.2]) #ошибка, т.к. нельзя перемешивать временной ряд
train_data = split[0]
test_data = split[1]


In [ ]:
train_data.show(5)

+--------------------+------+
|  vector_cols_scaled| CLOSE|
+--------------------+------+
|[0.0,4.5850947739...| 684.0|
|[0.00271064753340...| 684.9|
|[0.00292127217282...|684.53|
|[0.00318684237035...| 685.0|
|[0.00342493978882...| 686.0|
+--------------------+------+
only showing top 5 rows



In [ ]:
test_data.show(5)

+--------------------+------+
|  vector_cols_scaled| CLOSE|
+--------------------+------+
|[0.00279306587056...| 684.5|
|[0.00280222346358...| 684.0|
|[0.00315936959129...| 685.0|
|[0.00363556442824...|685.02|
|[0.00371798276540...| 685.9|
+--------------------+------+
only showing top 5 rows



#ML

In [ ]:
#linear regression
lr = LinearRegression(featuresCol='vector_cols_scaled',
                      labelCol='CLOSE')
model_lr = lr.fit(train_data)

#predict

In [ ]:
predicttion_df = model_lr.transform(test_data)

In [ ]:
predicttion_df.show(5)

+--------------------+------+-----------------+
|  vector_cols_scaled| CLOSE|       prediction|
+--------------------+------+-----------------+
|[0.00279306587056...| 684.5|683.7315749219374|
|[0.00280222346358...| 684.0|684.6255383018158|
|[0.00315936959129...| 685.0|684.8762369136288|
|[0.00363556442824...|685.02|685.0143478709875|
|[0.00371798276540...| 685.9|685.6996309926292|
+--------------------+------+-----------------+
only showing top 5 rows



#metrics

In [ ]:
summaryMetrics = model_lr.summary

In [ ]:
print(f'RMSE = {summaryMetrics.rootMeanSquaredError}')
print(f'r2 = {summaryMetrics.r2}')

RMSE = 0.4142005095519733
r2 = 0.9999979990807122


#spark pipeline, preprocessing + ML

In [ ]:
#объединение колонок в один веткор
assembler = VectorAssembler(inputCols = ['OPEN','MAX', 'MIN', 'VOLUME'],
                            outputCol='vector_cols')
#minmaxscaler
scaler = MinMaxScaler(inputCol='vector_cols',
                      outputCol='vector_cols_scaled') #масштабирвоание
#pipeline
pipeline = Pipeline(stages = [assembler, scaler, lr]) #параметры и шаги пайплайны
pipe_fit = pipeline.fit(data_spark)#обучение
dataset = pipe_fit.transform(data_spark) #добавялем к датасету новые колонки после пайпайна
dataset.show()

+----------+--------+------+------+------+------+------+--------------------+--------------------+-----------------+
|      DATE|    TIME|  OPEN|   MAX|   MIN| CLOSE|VOLUME|         vector_cols|  vector_cols_scaled|       prediction|
+----------+--------+------+------+------+------+------+--------------------+--------------------+-----------------+
|05.01.2004|10:30:00|681.05| 685.0|681.05| 684.0|  4833|[681.05,685.0,681...|[0.0,4.5850947739...|684.0651247265401|
|05.01.2004|10:31:00|684.79| 688.0|684.01| 686.0|  7025|[684.79,688.0,684...|[0.00342493978882...|686.6577191094937|
|05.01.2004|10:32:00| 685.0| 688.0| 685.0|685.12|  3956|[685.0,688.0,685....|[0.00361724924220...|687.2885893399935|
|05.01.2004|10:33:00|686.98| 687.0|686.02|686.02|  1105|[686.98,687.0,686...|[0.00543045265982...|686.2743311665555|
|05.01.2004|10:34:00| 687.0|688.68| 687.0| 688.0|  6270|[687.0,688.68,687...|[0.00544876784585...|688.2788853698247|
|05.01.2004|10:35:00| 688.5| 688.5|687.02| 687.9|   576|[688.5,6

#model save & load

In [ ]:
#save
model_lr.save('/content/spark_models/spark_lr.model')

In [ ]:
#load model
loaded_model_lr = LinearRegressionModel.load('/content/spark_models/spark_lr.model')

In [ ]:
loaded_model_predict = loaded_model_lr.transform(test_data)
loaded_model_predict.show(10)

+--------------------+------+-----------------+
|  vector_cols_scaled| CLOSE|       prediction|
+--------------------+------+-----------------+
|[0.00279306587056...| 684.5|683.7315749219374|
|[0.00280222346358...| 684.0|684.6255383018158|
|[0.00315936959129...| 685.0|684.8762369136288|
|[0.00363556442824...|685.02|685.0143478709875|
|[0.00371798276540...| 685.9|685.6996309926292|
|[0.00380955869559...| 686.5|686.1884795012386|
|[0.00407512889312...| 685.5|685.4943186689297|
|[0.00444143261385...| 685.9| 685.894312248694|
|[0.00453300854403...|684.01|684.5179015724975|
|[0.00453300854403...|686.01| 686.375884029278|
+--------------------+------+-----------------+
only showing top 10 rows



#Spark features engineering

In [ ]:
from pyspark.sql.functions import isnan, count, col, when, split
from pyspark.sql.types import *

In [ ]:
data_spark.show(5)

+----------+--------+------+------+------+------+------+
|      DATE|    TIME|  OPEN|   MAX|   MIN| CLOSE|VOLUME|
+----------+--------+------+------+------+------+------+
|05.01.2004|10:30:00|681.05| 685.0|681.05| 684.0|  4833|
|05.01.2004|10:31:00|684.79| 688.0|684.01| 686.0|  7025|
|05.01.2004|10:32:00| 685.0| 688.0| 685.0|685.12|  3956|
|05.01.2004|10:33:00|686.98| 687.0|686.02|686.02|  1105|
|05.01.2004|10:34:00| 687.0|688.68| 687.0| 688.0|  6270|
+----------+--------+------+------+------+------+------+
only showing top 5 rows



In [ ]:
data_spark_num = data_spark.select(['OPEN', 'MAX', 'MIN', 'CLOSE','VOLUME'])
data_spark_num.show(5)

+------+------+------+------+------+
|  OPEN|   MAX|   MIN| CLOSE|VOLUME|
+------+------+------+------+------+
|681.05| 685.0|681.05| 684.0|  4833|
|684.79| 688.0|684.01| 686.0|  7025|
| 685.0| 688.0| 685.0|685.12|  3956|
|686.98| 687.0|686.02|686.02|  1105|
| 687.0|688.68| 687.0| 688.0|  6270|
+------+------+------+------+------+
only showing top 5 rows



##считаем пропуски

In [ ]:
data_spark_num.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in data_spark_num.columns]).show()

+----+---+---+-----+------+
|OPEN|MAX|MIN|CLOSE|VOLUME|
+----+---+---+-----+------+
|   0|  0|  0|    0|     0|
+----+---+---+-----+------+



#Замена значений в столбце

In [ ]:
#Замена значений
data_spark_new = data_spark_num.withColumn('MAX',
                                           when(data_spark_num['MAX']==688.0, 999999).
                                           otherwise(data_spark_num['MAX']))
data_spark_new.show(5)

+------+--------+------+------+------+
|  OPEN|     MAX|   MIN| CLOSE|VOLUME|
+------+--------+------+------+------+
|681.05|   685.0|681.05| 684.0|  4833|
|684.79|999999.0|684.01| 686.0|  7025|
| 685.0|999999.0| 685.0|685.12|  3956|
|686.98|   687.0|686.02|686.02|  1105|
| 687.0|  688.68| 687.0| 688.0|  6270|
+------+--------+------+------+------+
only showing top 5 rows



##сумма колонок

In [ ]:
data_spark_new = data_spark_num.withColumn('SUM', data_spark_num['MAX'] + data_spark_num['MIN'])
data_spark_new.show(10)

+------+------+------+------+------+------------------+
|  OPEN|   MAX|   MIN| CLOSE|VOLUME|               SUM|
+------+------+------+------+------+------------------+
|681.05| 685.0|681.05| 684.0|  4833|           1366.05|
|684.79| 688.0|684.01| 686.0|  7025|           1372.01|
| 685.0| 688.0| 685.0|685.12|  3956|            1373.0|
|686.98| 687.0|686.02|686.02|  1105|           1373.02|
| 687.0|688.68| 687.0| 688.0|  6270|1375.6799999999998|
| 688.5| 688.5|687.02| 687.9|   576|           1375.52|
| 687.1| 687.1| 687.0| 687.0|  3330|            1374.1|
|686.99|686.99| 686.0| 686.0|  2679|           1372.99|
| 686.0| 686.5| 686.0| 686.0|  6227|            1372.5|
| 686.5| 686.5| 686.0| 686.0|  1350|            1372.5|
+------+------+------+------+------+------------------+
only showing top 10 rows



##Преобразование колонок в другой тип

In [ ]:
columns_list = data_spark_new.columns

for i in columns_list:
  data_spark_new_float = data_spark_new.withColumn(i, data_spark_new[i].cast(StringType()))

data_spark_new_float.show(10)

In [ ]:
data_spark_new_string_test = data_spark_new_float.withColumn('SUMM', data_spark_new_float['MAX'] + data_spark_new_float['MIN'])
data_spark_new_string_test.show(10)

In [ ]:
data_spark_new_string_test.dtypes

[('OPEN', 'double'),
 ('MAX', 'double'),
 ('MIN', 'double'),
 ('CLOSE', 'double'),
 ('VOLUME', 'int'),
 ('SUM', 'string'),
 ('SUMM', 'double')]